In [23]:
import json
import os
import sklearn
import numpy
import xgboost as xgb
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score
from collections import namedtuple
import sys
from math import log,tan,pi,radians
import pandas as pd
from sodapy import Socrata
from config import CHI_API_KEY
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [24]:
df = pd.read_csv("hoods.csv")

In [25]:
df.drop(["Unnamed: 0"], axis=1, inplace=True)

In [26]:
df['hood_label'].value_counts()
df = df[df.hood_label != 0]

In [27]:
df = df[df['hood_label'].notna()]
df = df[df['weather_condition'].notna()]
df = df[df['lighting_condition'].notna()]
df = df[df['crash_hour'].notna()]
df = df[df['crash_day_of_week'].notna()]
df = df[df['injuries_fatal'].notna()]

In [28]:
df = df[df['hood_label'].notna()]
df = df[df['weather_condition'].notna()]
df = df[df['lighting_condition'].notna()]

df['hit_and_run_i'] = df['hit_and_run_i'].fillna('N')

In [29]:
X = pd.get_dummies(df[['hood_label', "weather_condition", "lighting_condition", 'crash_hour', 'crash_day_of_week']])

In [30]:
values = array(df["hood_label"])

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)


onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
hl_onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

In [31]:
values = array(df["injuries_fatal"])

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)


onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
damage_onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

In [32]:
y = damage_onehot_encoded
print(X.shape)
print(y.shape)

(444969, 96)
(444969, 5)


In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

In [34]:
xgb_estimator = xgb.XGBClassifier(objective='binary:logistic')

In [35]:
multilabel_model = MultiOutputClassifier(xgb_estimator)

In [36]:
multilabel_model.fit(X_train, y_train)

MultiOutputClassifier(estimator=XGBClassifier(base_score=None, booster=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None, gamma=None,
                                              gpu_id=None,
                                              importance_type='gain',
                                              interaction_constraints=None,
                                              learning_rate=None,
                                              max_delta_step=None,
                                              max_depth=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              n_estimators=100, n_jobs=None,
                         

In [37]:
print('Accuracy on test data: {:.1f}%'.format(accuracy_score(y_test, multilabel_model.predict(X_test))*100))

Accuracy on test data: 99.9%


In [38]:
from joblib import dump, load
dump(multilabel_model, 'fatality.joblib') 

['fatality.joblib']